# Análisis y Tuning Avanzado de DeepForest
Este notebook aplica ajustes de color, parameters tuning de sliding window, anchor scales y NMS, y explora múltiples valores de patch_size y score_thresh para maximizar la detección de copas (30–50 px) en las imágenes de cítricos.

## 1. Clonar repositorio y configuración inicial

In [ ]:
import os
import git

# Parámetros de repositorio y carpetas
REPO_URL = 'https://github.com/maraosoc/citrus3-detector.git'
REPO_DIR = 'citrus3-detector'
DATA_DIR = os.path.join(REPO_DIR, 'data', 'samples')
RESULT_DIR = 'resultados_citrus_tuneado'

# Clonar si no existe localmente
if not os.path.exists(REPO_DIR):
    print('Clonando repositorio...')
    git.Repo.clone_from(REPO_URL, REPO_DIR)
else:
    print('Repositorio ya presente.')


## 2. Configuración de modelo y parámetros de tuning

In [ ]:
from deepforest import main

# Cargar modelo pre-entrenado
modelo = main.deepforest()
modelo.use_release()

# Ajustes globales para objetos pequeños (copas 30–50 px):
modelo.config['rpn_anchor_scales'] = [16, 32, 64, 128]  # anclas más pequeñas
modelo.config['nms_thresh'] = 0.6                     # menos supresión de cajas cercanas

# Valores de patch_size a explorar y solape del 50%
PATCH_SIZES = [256, 512, 800]
PATCH_OVERLAP = 0.5

# Umbrales de confianza a probar
UMBRAL_ES = [0.05, 0.1, 0.3]


## 3. Funciones auxiliares: IoU y métricas

In [ ]:
import numpy as np

def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0]); y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2]); y2 = min(box1[3], box2[3])
    inter = max(0, x2-x1) * max(0, y2-y1)
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter/union if union>0 else 0

def match_boxes(preds, truths, iou_thresh=0.4):
    matched = set(); tp=0
    for _, p in preds.iterrows():
        pbox = [p.xmin,p.ymin,p.xmax,p.ymax]
        for i, t in truths.iterrows():
            if i in matched: continue
            tbox = [t.xmin,t.ymin,t.xmax,t.ymax]
            if compute_iou(pbox,tbox)>=iou_thresh:
                tp+=1; matched.add(i); break
    fp = len(preds)-tp; fn = len(truths)-tp
    return tp, fp, fn

def compute_metrics(tp, fp, fn):
    precision = tp/(tp+fp) if tp+fp>0 else 0
    recall = tp/(tp+fn) if tp+fn>0 else 0
    f1 = 2*precision*recall/(precision+recall) if precision+recall>0 else 0
    return precision, recall, f1


## 4. Procesamiento con loops: resoluciones, patch_size y umbrales

In [ ]:
import cv2
import pandas as pd
from deepforest.visualize import plot_predictions

# Definir resoluciones a evaluar
RESOLUCIONES = {
    'original': None,
    'medio': (1033, 939),
    'pequeno': (516, 469)
}

os.makedirs(RESULT_DIR, exist_ok=True)
resumen = []

for img_file in os.listdir(DATA_DIR):
    if not img_file.lower().endswith(('.tif','.png','.jpg')): continue
    path = os.path.join(DATA_DIR, img_file)
    name = os.path.splitext(img_file)[0]
    # Leer BGR y convertir a RGB
    bgr = cv2.imread(path)
    if bgr is None: continue
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    # Cargar ground truth si existe
    gt_file = os.path.join(REPO_DIR,'data','annotations',f'{name}_gt.csv')
    truths = pd.read_csv(gt_file) if os.path.exists(gt_file) else None

    for res_name, size in RESOLUCIONES.items():
        img_res = cv2.resize(rgb, size, interpolation=cv2.INTER_AREA) if size else rgb.copy()
        for patch in PATCH_SIZES:
            modelo.config['patch_size'] = patch
            modelo.config['patch_overlap'] = PATCH_OVERLAP
            for thr in UMBRAL_ES:
                modelo.config['score_thresh'] = thr
                # Predicción con patch tuning explícito
                boxes = modelo.predict_image(
                    image=img_res,
                    patch_size=patch,
                    patch_overlap=PATCH_OVERLAP,
                    return_plot=False
                )
                boxes = boxes[boxes.score>=thr]

                # Métricas
                if truths is not None:
                    tp, fp, fn = match_boxes(boxes, truths)
                    precision, recall, f1 = compute_metrics(tp, fp, fn)
                else:
                    tp=fp=fn=None; precision=recall=f1=None

                # Guardar resultados
                boxes.to_csv(os.path.join(
                    RESULT_DIR, f'cajas_{res_name}_p{patch}_th{thr}_{name}.csv'
                ), index=False)
                # Imagen anotada
                ann = plot_predictions(img_res, boxes)
                ann_bgr = cv2.cvtColor(ann, cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join(
                    RESULT_DIR, f'annotated_{res_name}_p{patch}_th{thr}_{name}.png'
                ), ann_bgr)
                # Data al resumen
                resumen.append({
                    'Imagen': name,
                    'Resolución': res_name,
                    'Patch_size': patch,
                    'Score_thresh': thr,
                    'Detections': len(boxes),
                    'TP': tp, 'FP': fp, 'FN': fn,
                    'Precision': precision, 'Recall': recall, 'F1': f1
                })
df = pd.DataFrame(resumen)
df.to_csv(os.path.join(RESULT_DIR,'resumen_tuneado.csv'), index=False)


## 5. Resultados
- Todos los CSV de predicciones y las imágenes anotadas están en `resultados_citrus_tuneado`.
- El resumen con todas las combinaciones está en `resumen_tuneado.csv`.

Puedes descargar el notebook tuneado aquí:
[Descargar Notebook Tuneado](sandbox:/mnt/data/analisis_de_arboles_tuneado.ipynb)